In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM
from google.colab import files
from google.colab import drive
import numpy as np
#@title Import Statements
!pip install PyGithub

# Package Imports
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
from pathlib import Path # The Path class

# For the more advanced requests
import base64
import os
import sys
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass


     |████████████████████████████████| 296kB 16.9MB/s 


In [ ]:
#@title Function Definitions
# Recursively Import the Data (AUTOMATIC)

def _decode_and_write(file__, path_):
    data = file__.decoded_content
    data = data.decode('utf-8')[1:]
    with open(path_, 'w') as writefile:
        writefile.write(data) 
    data = data.splitlines()
    data_rows = []
    for count, word in enumerate(data):
        if count>0:
            data_rows.append(word.split(','))
    data = pd.DataFrame(data_rows)
    data = data.to_numpy()
    return data


def import_github(path_name="AllLyrics.txt"):
    """
    Function for importing the github file
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()
    print("Importing Github cleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    RAP_DATA = []
    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the Lyrics
            RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
    
    temp_path = Path(path_name)
    if temp_path.is_file(): 
        if os.stat(path_name).st_size == 0:
            write_bool2 = True
        else: 
            write_bool2 = False
    else: 
        write_bool2 = True
    
    if write_bool2: 
        for lyric in RAP_DATA: 
            try:
                with open(path_name, 'w') as writefile: 
                    writefile.write(lyric)
            except: 
                print("Error, file moved/deleted during write")
        print("{} is now up to date!".format(path_name))
    else: 
        print("{} is already up to date!".format(path_name))
    
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 

        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        path = path[title_start + 6:title_start + title_len + 4]

        print("Writing file {} {}".format(counter, path))
        temp_path = Path(path)
        if temp_path.is_file():
            with open(path,'w'): pass # Cheeky way to clear the file if it exists
        
        # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy array 
        data = file_.decoded_content
        data = data.decode('utf-8')[1:]

        with open(path, 'w') as writefile:
            writefile.write(data) 
        print("All files now up to date!")


def update_github(write_bool=False, path_name="AllLyrics.txt"):
    """
    Function for updating the github file, by cleaning the lyrics, optional write to txt file. 
    write_bool: bool
    path_name: str
    output: None
    """
    g = Github(getpass("Enter your PAT key ")) # Enter your PAT Key.
    username = "MikeMNelhams"
    main_branch_bool = input("Main Branch: Yes or No? ")
    yes_synonyms = ["yes", "y", "yh", "1", "true"]
    if main_branch_bool.lower() in yes_synonyms: 
        branch = "master" 
    else: 
        branch = "PROTOTYPE"

    user = g.get_user(username)
    r_proj_clone = 0
    for repo in g.get_user().get_repos():
        if repo.name == "ai-group-project-Team-JMJM":
            r_proj_clone = repo
            break
        # To see all the available attributes and methods
        print(dir(repo))
    
    if not r_proj_clone:
        print("ai-group-project-Team-JMJM not found")
        sys.exit()

    print("Importing editing csv files...")

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    contents = r_proj_clone.get_contents("RapLyrics/Other", ref=branch)
    for counter, file_ in enumerate(contents):
        path = file_.path 
        path = str(path)
        title_start = path.find('Other')
        title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len + 4]
        print("Writing file {} {}".format(counter, name))
        if name.lower() == "censors.csv":
            censors = _decode_and_write(file_, path)
        elif name.lower() == "capitals.csv":
            capitals = _decode_and_write(file_, path)
        else: 
            _decode_and_write(file_, path)
    print("All editing csv files are up to date!")

    print("Importing Github uncleaned text files...")
    contents = r_proj_clone.get_contents("RapLyrics/UNCLEAN", ref=branch)

    RAP_DATA = []
    rap_lyric_names = []

    for file_ in contents:
        path = file_.path
        path = str(path) 
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('UNCLEAN')
            title_len = path[title_start:].find('.')
            name = path[title_start + 8:title_start + title_len]
            if name[-2:] == 'UC':
                name = name[:-2]
            rap_lyric_names.append(name) 

        # Append the Lyrics
        RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
        
    # Remove the \ufeff at the beginning O(n)
    for count, lyric in enumerate(RAP_DATA): 
        RAP_DATA[count] = lyric[1:]

    # Censor the profanities O(n*m + n*m2) m > m2 xor m2 > m
    for count in range(len(RAP_DATA)): 
        for i in range(len(censors[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))
        for i in range(len(capitals[0:])):
            RAP_DATA[count] = RAP_DATA[count].replace(str(capitals[i, 0]), str(capitals[i, 1]))

    contents = r_proj_clone.get_contents("RapLyrics/CLEAN", ref=branch)
    cleaned_names = []
    for counter, file_ in enumerate(contents):
        path = file_.path
        path = str(path) 
        print("File {} ".format(counter + 1) + path)
        # Only choose the .txt files
        if path[-4:] == '.txt':
            # Append the name
            title_start = path.find('CLEAN')
            title_len = path[title_start:].find('.')
        name = path[title_start + 6:title_start + title_len]
        if name[-2:] == 'CL':
            name = name[:-2]
        cleaned_names.append(name) 

    # ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH' to avoid overwriting import changes
    # If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
    # If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
    # print(rap_lyric_names)
    # print(cleaned_names)
    print("Commiting files to github...")
    for counter, new_name in enumerate(rap_lyric_names): 
        if new_name in cleaned_names: 
            duplicate = r_proj_clone.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref=branch)
            r_proj_clone.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch=branch)
        else:
            r_proj_clone.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch=branch)

    if write_bool: 
        print("Writing text file to: {}".format(path_name))
        with open(path_name, 'w') as writefile:
            for lyric in RAP_DATA:
                writefile.write(lyric)

In [ ]:
# Import all of Mike's lyrics. PATKEY: 5ae2446bd5828c9e27deb3865118d9e783aa6e15
import_github()
Text = open("AllLyrics.txt", "r").read()
# turn text to lower case to reduce vocabulary
Text = Text.lower()
with open("AllLyrics.txt", "r") as f:
    content = f.readlines()
# Defined content in case we need to split each line/bar into strings
content = [x.strip() for x in content] 
Vocabulary = ''.join([i for i in Text if not i.isdigit()]).replace("\n"," ").split(' ')


Enter your PAT key ··········
Main Branch: Yes or No? Yes
Importing Github cleaned text files...
AllLyrics.txt is now up to date!
Writing file 0 capitals.csv
All files now up to date!
Writing file 1 censors.csv
All files now up to date!


In [ ]:
# List of all unique vocabulary in alphabetical order
words = sorted(list(set(Vocabulary)))
print(words)
print(len(words))
# Need to create a dictionary as LTSM model needs numbers, not words (will word this better)
intdict = { i : words[i] for i in range(len(words))}
print(intdict)
# Need to reverse this at the end to reverse numbers back into words
charsdict = { words[i] : i for i in range(len(words))}
print(charsdict)

['', '#', 'a', 'aaalll', 'aand', 'able', 'abortion', 'about', 'above', 'abuser', 'abusers', 'ac', 'accept', 'acclaimed', 'account', 'accountant', 'ace', 'across', 'acs', 'act', 'actin', 'acting', 'action', 'actors', 'acts', 'actually', 'adapt', 'add', 'address', 'admit', 'adolescents', 'advance', 'advanced', 'advancin', 'advancing', 'adventure', 'aes', 'affect', 'afraid', 'africans', 'after', 'again', 'against', 'age', 'agenda', 'agents', 'agreed', 'ahh', 'aids', 'aight', 'aim', 'aimin', 'aint', 'air', 'airport', 'aiy', 'ak', 'aka', 'akon', 'alarm', 'alberta', 'album', 'alcoholic', 'ale', 'alfred', 'alibi', 'alive', 'alize', 'all', 'alligators', 'allow', 'allowed', 'almost', 'alone', 'along', 'alot', 'already', 'alright', 'also', 'always', 'am', 'amateurs', 'americas', 'amnesia', 'amount', 'amped', 'amtrak', 'amuck', 'an', 'and', 'ands', 'angel', 'animal', 'animals', 'ankles', 'annihilate', 'announced', 'anoint', 'anotha', 'another', 'answer', 'antartic', 'anthem', 'any', 'anybody', 'a

In [ ]:
def word_count(lyrics):
  a = {}
  for word in Vocabulary:
    if word in a:
      a[word] += 1
    else:
      a[word] = 1
  return a
word_dict = word_count(Vocabulary)
sort_dict = sorted(word_dict.items(), key = lambda x: x[1], reverse = True)
# Top 20 words
sort_dict1 = sort_dict[:20]
sort_dict1

[('the', 2126),
 ('i', 1421),
 ('you', 1323),
 ('to', 1072),
 ('and', 904),
 ('my', 699),
 ('a', 692),
 ('in', 580),
 ('me', 562),
 ('on', 500),
 ('your', 494),
 ('that', 431),
 ('it', 423),
 ('im', 394),
 ('what', 388),
 ('with', 368),
 ('is', 353),
 ('like', 350),
 ('for', 330),
 ('ninja', 330)]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(content)
sequences1 = tokenizer.texts_to_sequences(content)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)


5064


In [ ]:
word_index = tokenizer.word_index
word_index['#']

KeyError: ignored

In [ ]:
reverse_word_index = dict([(v,k) for (k,v) in word_index.items()])


In [ ]:
input_sequences = []
for line in content:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)
tokenizer.texts_to_sequences([content[0]])

[[81, 13, 49, 22, 7, 545]]

In [ ]:
MAX_SEQ_LEN = max([len(x) for x in input_sequences])
MAX_SEQ_LEN

66

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen = MAX_SEQ_LEN, padding = 'pre'))

xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes = vocab_size)
xs

array([[  0,   0,   0, ...,   0,   0,  81],
       [  0,   0,   0, ...,   0,  81,  13],
       [  0,   0,   0, ...,  81,  13,  49],
       ...,
       [  0,   0,   0, ...,   0,  86,  11],
       [  0,   0,   0, ...,  86,  11, 317],
       [  0,   0,   0, ...,  11, 317, 116]], dtype=int32)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional, Dropout
model = Sequential([
    Embedding(vocab_size, 256, input_length = MAX_SEQ_LEN - 1),
    Bidirectional(LSTM(128, return_sequences = True)),
    Bidirectional(LSTM(128)),
    Dropout(0.2),
    Dense(vocab_size, activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 65, 256)           1296384   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 65, 256)           394240    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5064)              1301448   
Total params: 3,386,312
Trainable params: 3,386,312
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(xs, ys, epochs = 200, batch_size = 256)

Epoch 1/200
161/161 [==============================] - 14s 62ms/step - loss: 7.0910 - accuracy: 0.0449
Epoch 2/200
161/161 [==============================] - 10s 61ms/step - loss: 6.2835 - accuracy: 0.0506
Epoch 3/200
161/161 [==============================] - 10s 61ms/step - loss: 6.1050 - accuracy: 0.0612
Epoch 4/200
161/161 [==============================] - 10s 61ms/step - loss: 5.9293 - accuracy: 0.0725
Epoch 5/200
161/161 [==============================] - 10s 61ms/step - loss: 5.8252 - accuracy: 0.0784
Epoch 6/200
161/161 [==============================] - 10s 62ms/step - loss: 5.7028 - accuracy: 0.0851
Epoch 7/200
161/161 [==============================] - 10s 62ms/step - loss: 5.5492 - accuracy: 0.0994
Epoch 8/200
161/161 [==============================] - 10s 61ms/step - loss: 5.4141 - accuracy: 0.1136
Epoch 9/200
161/161 [==============================] - 10s 61ms/step - loss: 5.3113 - accuracy: 0.1282
Epoch 10/200
161/161 [==============================] - 10s 61ms/step - l

In [ ]:
!mkdir -p lstm-working
model.save('lstm-working')
!ls lstm-working

INFO:tensorflow:Assets written to: lstm-working/assets


INFO:tensorflow:Assets written to: lstm-working/assets


assets	saved_model.pb	variables


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/lstm-working')
new_model.summary()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f5c2f07ecc0>>

In [ ]:
def generateraplyrics():
  seed_text  = random.choice(content)
  next_words = random.randint(6,12)
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen = MAX_SEQ_LEN - 1, padding = 'pre')
  predicted  = model.predict_classes(token_list, verbose = 0)
  lyrics = reverse_word_index[predicted[0]]
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([lyrics])[0]
      token_list = pad_sequences([token_list], maxlen = MAX_SEQ_LEN - 1, padding = 'pre')
      predicted  = model.predict_classes(token_list, verbose = 0)
      lyrics += ' ' + reverse_word_index[predicted[0]]
  return lyrics

In [ ]:
generateraplyrics()

With your hat to the back talkin bout the gats in your raps


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'i got the funk flow to make your drawers drop'

In [ ]:
rap

['i got the funk flow to make your drawers drop slow like',
 'til they piss the poop uhh uhh she knew i can get a',
 'where the true players at that ninja',
 'and i just love your flashy ways uhhuh to the ra t yall',
 'on the cellycelly with my homeboy lance frickin',
 'yeah yeah youll take a little bum across its the tristate unloaded',
 'do you ever ask yourself when are they gonna stop making us dance',
 'what you want ninja what you what you want ninja aye',
 'unloaded open the drug to stop you on the gutter',
 'what you want ninja what you what you want ninja aye yo',
 'baby sing relax and feel up i']